In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import time
import random

In [4]:
# 获取歌曲名称
new_song_names = pd.read_csv("all_songs.csv")['songs'].to_list()
old_song_names = pd.read_csv("song_infos.csv")['歌曲名称'].to_list()
add_song_names = list(set(new_song_names)-set(old_song_names))

In [5]:
# 检查用
add_song_names

['A Day At A Time',
 "You're My Superhero",
 'Help!',
 '一千年后记得我',
 '我们很好',
 '新年到',
 '梦想的翅膀',
 '绝不绝',
 'Better With You',
 '全面开战',
 '你還不知道?',
 '사랑해요只对你说',
 'Positive Procrastination',
 '曙光']

In [6]:
song_infos = []

url_baike = "https://baike.baidu.com"

# 四种情况
# case 1:正确找到对应歌曲百科
# case 2:找到多义词的百科界面-->需跳转到多义词的列表界面-->找到对应歌曲百科
# case 3:找到多义词的列表界面-->找到对应歌曲百科
# case 4:找不到百科

# 找到资料的前一个a标签
def is_info_pre_a(tag):
    return tag.has_attr('name') and tag['name'] == 'lemma-summary'

pattern = r'^共\d+个义项'
# 若有多义词，则必存在指向多义词的a标签
def is_multi_a(tag):
    return tag.has_attr('href') and re.search(pattern=pattern, string=tag.text)

# 从多义词列表中找到歌曲链接
def is_target_a(tag):
    return tag.has_attr('href') and "林俊杰" in tag.text

# case 1
def get_baike(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "lxml")
    try:
        d = soup.find(is_info_pre_a).next_sibling
        if "林俊杰" in d.get_text():
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")
        else:
            # case 2
            d = soup.find(is_multi_a)
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")
            # 进入case 3
            d = soup.find(is_target_a)
            if d is None:
                # 说明找不到该歌曲对应百科
                print("找不到对应百科")
            else:
                res = requests.get(url=url_baike+d['href'])
                soup = BeautifulSoup(res.text, "lxml")

                d = soup.find(is_info_pre_a).next_sibling
                song_infos.append([name, d.get_text()])
                print("添加歌曲信息成功")
    except:
        # case 3 or case 4
        # 如果能找到有林俊杰的a标签则是case 3
        d = soup.find(is_target_a)
        if d is None:
            # 说明找不到该歌曲对应百科
            print("找不到对应百科")
        else:
            res = requests.get(url=url_baike+d['href'])
            soup = BeautifulSoup(res.text, "lxml")

            d = soup.find(is_info_pre_a).next_sibling
            song_infos.append([name, d.get_text()])
            print("添加歌曲信息成功")

num = 1
for name in add_song_names:
    print("["+str(num) + "/" + str(len(add_song_names))+"] "+name+":")
    get_baike(url=url_baike+"/item/"+name)

    num += 1
    n = random.randint(0, 10)
    time.sleep(n * 0.1)
    



[1/14] A Day At A Time:
找不到对应百科
[2/14] You're My Superhero:
找不到对应百科
[3/14] Help!:
找不到对应百科
[4/14] 一千年后记得我:
添加歌曲信息成功
[5/14] 我们很好:
添加歌曲信息成功
[6/14] 新年到:
添加歌曲信息成功
[7/14] 梦想的翅膀:
找不到对应百科
[8/14] 绝不绝:
添加歌曲信息成功
[9/14] Better With You:
找不到对应百科
[10/14] 全面开战:
找不到对应百科
[11/14] 你還不知道?:
找不到对应百科
[12/14] 사랑해요只对你说:
找不到对应百科
[13/14] Positive Procrastination:
找不到对应百科
[14/14] 曙光:
找不到对应百科


In [7]:
song_infos

[['一千年后记得我',
  '歌曲《一千年后记得我》，由林俊杰演唱，林秋离填词，林俊杰谱曲，收录于林俊杰2013年3月13日正式发行的第十张个人专辑《因你而在》中的隐藏曲目 [2]'],
 ['我们很好', '《我们很好》是林俊杰于2019年6月13日发布的单曲。该歌曲由葛大为填词，林俊杰谱曲，为电影《少年的你》的主题曲 [1-2]。'],
 ['新年到',
  '由众歌手首次大合唱演绎的《新年到》, 演绎歌手有JJ林俊杰，Rynn林宇中，李志清，BY2， Wil黄威尔， Phoebe菲比，黄启铭，汤小康，曾若冰， CHIN ，满江浤，陈泽耀。 [1]'],
 ['绝不绝', '《绝不绝》是林俊杰 /无畏契约共同演唱歌曲，唱片公司：JFJ Productions，发行时间：2024-07-20 [1]。']]

In [10]:
# 将歌曲信息添加进文件中保存
add_df = pd.DataFrame(columns=["歌曲名称","歌曲信息"], data=song_infos)
old_df = pd.read_csv("song_infos.csv", index_col=0)
new_df = pd.concat([old_df, add_df], ignore_index=True)
new_df.to_csv("song_infos.csv")

In [9]:
new_df

,歌曲名称,歌曲信息
0,The Show,《The Show》是由Steve Aoki、Carl Ryden、James Daniel...
1,愿与愁,《愿与愁》是林俊杰演唱的歌曲，由小寒作词，林俊杰作曲，于2023年4月14日以单曲形式发行 ...
2,逆光白,《逆光白》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于林俊杰于2023年4月21日发行...
3,孤独娱乐,《孤独娱乐》是由易家扬作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑...
4,梦不凌乱,《梦不凌乱》是由方文山作词，林俊杰作曲并演唱的一首歌曲，收录于2023年4月21日发行的专辑...
...,...,...
291,不懂,《不懂》是林俊杰演唱的歌曲，由张思尔填词，林俊杰谱曲，收录在专辑《乐行者》中。
292,一千年后记得我,歌曲《一千年后记得我》，由林俊杰演唱，林秋离填词，林俊杰谱曲，收录于林俊杰2013年3月13...
293,我们很好,《我们很好》是林俊杰于2019年6月13日发布的单曲。该歌曲由葛大为填词，林俊杰谱曲，为电影...
294,新年到,"由众歌手首次大合唱演绎的《新年到》, 演绎歌手有JJ林俊杰，Rynn林宇中，李志清，BY2，..."
